In [1]:
# Libraries
import numpy as np
import pandas as pd
import re

In [2]:
import sys
sys.path.append("..")
from src.cleaning import *

# Primera visualización rápida

Cargamos datos de formato CSV

In [3]:
attacks = pd.read_csv("../data/attacks.csv", encoding = "ISO-8859-1")

Visualizo las primeras y últimas filas para echar una ojeada a los datos.

In [4]:
attacks.head(2)

Case Number         Date    Year        Type Country        Area  \
0  2018.06.25  25-Jun-2018  2018.0     Boating     USA  California   
1  2018.06.18  18-Jun-2018  2018.0  Unprovoked     USA     Georgia   

                         Location  Activity             Name Sex   ...  \
0     Oceanside, San Diego County  Paddling      Julie Wolfe    F  ...   
1  St. Simon Island, Glynn County  Standing  Adyson McNeely     F  ...   

      Species           Investigator or Source                     pdf  \
0  White shark                R. Collier, GSAF    2018.06.25-Wolfe.pdf   
1          NaN  K.McMurray, TrackingSharks.com  2018.06.18-McNeely.pdf   

                                        href formula  \
0  http://sharkattackfile.net/spreadsheets/pdf_di...   
1  http://sharkattackfile.net/spreadsheets/pdf_di...   

                                                href Case Number.1  \
0  http://sharkattackfile.net/spreadsheets/pdf_di...    2018.06.25   
1  http://sharkattackfile.net/spreadsheets/pdf_di...    2018.06.18   

  Case Number.2 original order Unnamed: 22 Unnamed: 23  
0    2018.06.25         6303.0         NaN         NaN  
1    2018.06.18         6302.0         NaN         NaN  

[2 rows x 24 columns]

In [5]:
attacks.tail(2)

Case Number Date  Year Type Country Area Location Activity Name Sex   \
25721         NaN  NaN   NaN  NaN     NaN  NaN      NaN      NaN  NaN  NaN   
25722          xx  NaN   NaN  NaN     NaN  NaN      NaN      NaN  NaN  NaN   

       ... Species  Investigator or Source  pdf href formula href  \
25721  ...      NaN                    NaN  NaN          NaN  NaN   
25722  ...      NaN                    NaN  NaN          NaN  NaN   

      Case Number.1 Case Number.2 original order Unnamed: 22 Unnamed: 23  
25721           NaN           NaN            NaN         NaN         NaN  
25722           NaN           NaN            NaN         NaN         NaN  

[2 rows x 24 columns]

Visualizo también una muestra aleatoria.

In [6]:
attacks.sample(2)

Case Number Date  Year Type Country Area Location Activity Name Sex   \
12939         NaN  NaN   NaN  NaN     NaN  NaN      NaN      NaN  NaN  NaN   
17163         NaN  NaN   NaN  NaN     NaN  NaN      NaN      NaN  NaN  NaN   

       ... Species  Investigator or Source  pdf href formula href  \
12939  ...      NaN                    NaN  NaN          NaN  NaN   
17163  ...      NaN                    NaN  NaN          NaN  NaN   

      Case Number.1 Case Number.2 original order Unnamed: 22 Unnamed: 23  
12939           NaN           NaN            NaN         NaN         NaN  
17163           NaN           NaN            NaN         NaN         NaN  

[2 rows x 24 columns]

# Mis HIPOTESIS.

1_ USA es el pais donde más accidentes de tiburón se producen.

2_ La mayor parte de los accidentes son no provocados.

3_ Fishing y Surfing; mayores causas de accidentes.

4_ Mueren más hombres que mujeres.

# Análisis exploratorio del DataFrame

Evaluar forma, info y Nan del Data Frame. Para este caso no merece la pena hacer un describe. 

In [7]:
attacks.shape

(25723, 24)

In [8]:
attacks.info

<bound method DataFrame.info of       Case Number         Date    Year        Type    Country  \
0      2018.06.25  25-Jun-2018  2018.0     Boating        USA   
1      2018.06.18  18-Jun-2018  2018.0  Unprovoked        USA   
2      2018.06.09  09-Jun-2018  2018.0     Invalid        USA   
3      2018.06.08  08-Jun-2018  2018.0  Unprovoked  AUSTRALIA   
4      2018.06.04  04-Jun-2018  2018.0    Provoked     MEXICO   
...           ...          ...     ...         ...        ...   
25718         NaN          NaN     NaN         NaN        NaN   
25719         NaN          NaN     NaN         NaN        NaN   
25720         NaN          NaN     NaN         NaN        NaN   
25721         NaN          NaN     NaN         NaN        NaN   
25722          xx          NaN     NaN         NaN        NaN   

                  Area                        Location     Activity  \
0           California     Oceanside, San Diego County     Paddling   
1              Georgia  St. Simon Island, Gly

In [9]:
attacks.isnull().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

Hago comprbación de DATOS NUMËRICOS y CATEGÓRICOS

In [10]:
# DATOS NUMÉRICOS
numericas = attacks._get_numeric_data()
numericas.head(3)

Year  original order
0  2018.0          6303.0
1  2018.0          6302.0
2  2018.0          6301.0

In [11]:
# DATOS CATEGÓRICOS
attacks.select_dtypes(include="object").head(3)

Case Number         Date        Type Country        Area  \
0  2018.06.25  25-Jun-2018     Boating     USA  California   
1  2018.06.18  18-Jun-2018  Unprovoked     USA     Georgia   
2  2018.06.09  09-Jun-2018     Invalid     USA      Hawaii   

                         Location  Activity             Name Sex  Age  ...  \
0     Oceanside, San Diego County  Paddling      Julie Wolfe    F  57  ...   
1  St. Simon Island, Glynn County  Standing  Adyson McNeely     F  11  ...   
2                    Habush, Oahu   Surfing      John Denges    M  48  ...   

            Time     Species           Investigator or Source  \
0          18h00  White shark                R. Collier, GSAF   
1  14h00  -15h00          NaN  K.McMurray, TrackingSharks.com   
2          07h45          NaN  K.McMurray, TrackingSharks.com   

                      pdf                                       href formula  \
0    2018.06.25-Wolfe.pdf  http://sharkattackfile.net/spreadsheets/pdf_di...   
1  2018.06.18-McNeely.pdf  http://sharkattackfile.net/spreadsheets/pdf_di...   
2   2018.06.09-Denges.pdf  http://sharkattackfile.net/spreadsheets/pdf_di...   

                                                href Case Number.1  \
0  http://sharkattackfile.net/spreadsheets/pdf_di...    2018.06.25   
1  http://sharkattackfile.net/spreadsheets/pdf_di...    2018.06.18   
2  http://sharkattackfile.net/spreadsheets/pdf_di...    2018.06.09   

  Case Number.2 Unnamed: 22 Unnamed: 23  
0    2018.06.25         NaN         NaN  
1    2018.06.18         NaN         NaN  
2    2018.06.09         NaN         NaN  

[3 rows x 22 columns]

Renombro COLUMNAS. Elimino mayúsculas y espacios.

Nota; más adelante me di cuenta que tendría que renombrar de nuevo la columna "fata(y/n)" para poder trabajar sobre ella. Haré referncia a ello más adelante.

In [12]:
attacks_copia = attacks.copy()

In [13]:
# Saco los 5 primeros elementos.
columnas = list(attacks.columns)
columnas[:5]

['Case Number', 'Date', 'Year', 'Type', 'Country']

In [14]:
diccio_nuevas = {columna: columna.replace(" ", "_").lower() for columna in columnas}

In [15]:
diccio_nuevas

{'Case Number': 'case_number',
 'Date': 'date',
 'Year': 'year',
 'Type': 'type',
 'Country': 'country',
 'Area': 'area',
 'Location': 'location',
 'Activity': 'activity',
 'Name': 'name',
 'Sex ': 'sex_',
 'Age': 'age',
 'Injury': 'injury',
 'Fatal (Y/N)': 'fatal_(y/n)',
 'Time': 'time',
 'Species ': 'species_',
 'Investigator or Source': 'investigator_or_source',
 'pdf': 'pdf',
 'href formula': 'href_formula',
 'href': 'href',
 'Case Number.1': 'case_number.1',
 'Case Number.2': 'case_number.2',
 'original order': 'original_order',
 'Unnamed: 22': 'unnamed:_22',
 'Unnamed: 23': 'unnamed:_23'}

In [16]:
attacks_renombrado = attacks.rename(columns=diccio_nuevas)
attacks_renombrado.sample()

case_number date  year type country area location activity name sex_  \
23235         NaN  NaN   NaN  NaN     NaN  NaN      NaN      NaN  NaN  NaN   

       ... species_ investigator_or_source  pdf href_formula href  \
23235  ...      NaN                    NaN  NaN          NaN  NaN   

      case_number.1 case_number.2 original_order unnamed:_22 unnamed:_23  
23235           NaN           NaN            NaN         NaN         NaN  

[1 rows x 24 columns]

In [17]:
attacks.rename(columns=diccio_nuevas, inplace=True)

In [18]:
attacks.head(2)

case_number         date    year        type country        area  \
0  2018.06.25  25-Jun-2018  2018.0     Boating     USA  California   
1  2018.06.18  18-Jun-2018  2018.0  Unprovoked     USA     Georgia   

                         location  activity             name sex_  ...  \
0     Oceanside, San Diego County  Paddling      Julie Wolfe    F  ...   
1  St. Simon Island, Glynn County  Standing  Adyson McNeely     F  ...   

      species_          investigator_or_source                     pdf  \
0  White shark                R. Collier, GSAF    2018.06.25-Wolfe.pdf   
1          NaN  K.McMurray, TrackingSharks.com  2018.06.18-McNeely.pdf   

                                        href_formula  \
0  http://sharkattackfile.net/spreadsheets/pdf_di...   
1  http://sharkattackfile.net/spreadsheets/pdf_di...   

                                                href case_number.1  \
0  http://sharkattackfile.net/spreadsheets/pdf_di...    2018.06.25   
1  http://sharkattackfile.net/spreadsheets/pdf_di...    2018.06.18   

  case_number.2 original_order unnamed:_22 unnamed:_23  
0    2018.06.25         6303.0         NaN         NaN  
1    2018.06.18         6302.0         NaN         NaN  

[2 rows x 24 columns]

Compruebo los datos que no son Nan en la columna unnamed:_23 para ver si hay algún dato relevante.

In [19]:

attacks[attacks["unnamed:_23"].notna()]

case_number                  date    year        type  \
4415    1952.03.30           30-Mar-1952  1952.0  Unprovoked   
5840  1878.09.14.R  Reported 14-Sep-1878  1878.0    Provoked   

                   country         area                    location  \
4415  NETHERLANDS ANTILLES      Curacao                         NaN   
5840                   USA  Connecticut  Branford, New Haven County   

                                             activity              name sex_  \
4415  Went to aid of child being menaced by the shark       A.J. Eggink    M   
5840                                          Fishing  Captain Pattison    M   

      ...                                           species_  \
4415  ...  Bull shark, 2.7 m [9'] was captured & dragged ...   
5840  ...                                                NaN   

                                 investigator_or_source  \
4415  J. Randall, p.352 in Sharks & Survival; H.D. B...   
5840                       St. Joseph Herald, 9/14/1878   

                            pdf  \
4415      1952.03.30-Eggink.pdf   
5840  1878.09.14.R-Pattison.pdf   

                                           href_formula  \
4415  http://sharkattackfile.net/spreadsheets/pdf_di...   
5840  http://sharkattackfile.net/spreadsheets/pdf_di...   

                                                   href case_number.1  \
4415  http://sharkattackfile.net/spreadsheets/pdf_di...    1952.03.30   
5840  http://sharkattackfile.net/spreadsheets/pdf_di...  1878.09.14.R   

     case_number.2 original_order unnamed:_22      unnamed:_23  
4415    1952.03.30         1888.0         NaN           Teramo  
5840  1878.09.14.R          463.0         NaN  change filename  

[2 rows x 24 columns]

Elimino los paréntesis de la columna fatal para poder ver más adelante las muertes.

In [20]:
columnas_1 = list(attacks.columns)

In [21]:
diccio_nuevas_1 = {columna: columna.replace("(y/n)", "") for columna in columnas_1}

In [22]:
diccio_nuevas_1

{'case_number': 'case_number',
 'date': 'date',
 'year': 'year',
 'type': 'type',
 'country': 'country',
 'area': 'area',
 'location': 'location',
 'activity': 'activity',
 'name': 'name',
 'sex_': 'sex_',
 'age': 'age',
 'injury': 'injury',
 'fatal_(y/n)': 'fatal_',
 'time': 'time',
 'species_': 'species_',
 'investigator_or_source': 'investigator_or_source',
 'pdf': 'pdf',
 'href_formula': 'href_formula',
 'href': 'href',
 'case_number.1': 'case_number.1',
 'case_number.2': 'case_number.2',
 'original_order': 'original_order',
 'unnamed:_22': 'unnamed:_22',
 'unnamed:_23': 'unnamed:_23'}

In [23]:
attacks_renombrado = attacks.rename(columns=diccio_nuevas_1)
attacks_renombrado.sample()

case_number         date    year        type country        area  \
324  2015.12.21.a  21-Dec-2015  2015.0  Unprovoked  BRAZIL  Pernambuco   

               location      activity                     name sex_  ...  \
324  Fernano de Noronha  Scuba diving  Márcio de Castro Palma     M  ...   

                species_ investigator_or_source                      pdf  \
324  Tiger shark, 1.5 m    Fox News, 12/22/2015  2015.12.21.a-Brazil.pdf   

                                          href_formula  \
324  http://sharkattackfile.net/spreadsheets/pdf_di...   

                                                  href case_number.1  \
324  http://sharkattackfile.net/spreadsheets/pdf_di...  2015.12.21.a   

    case_number.2 original_order unnamed:_22 unnamed:_23  
324  2015.12.21.a         5979.0         NaN         NaN  

[1 rows x 24 columns]

In [24]:
attacks.rename(columns=diccio_nuevas_1, inplace=True)

In [25]:
attacks.head(2)

case_number         date    year        type country        area  \
0  2018.06.25  25-Jun-2018  2018.0     Boating     USA  California   
1  2018.06.18  18-Jun-2018  2018.0  Unprovoked     USA     Georgia   

                         location  activity             name sex_  ...  \
0     Oceanside, San Diego County  Paddling      Julie Wolfe    F  ...   
1  St. Simon Island, Glynn County  Standing  Adyson McNeely     F  ...   

      species_          investigator_or_source                     pdf  \
0  White shark                R. Collier, GSAF    2018.06.25-Wolfe.pdf   
1          NaN  K.McMurray, TrackingSharks.com  2018.06.18-McNeely.pdf   

                                        href_formula  \
0  http://sharkattackfile.net/spreadsheets/pdf_di...   
1  http://sharkattackfile.net/spreadsheets/pdf_di...   

                                                href case_number.1  \
0  http://sharkattackfile.net/spreadsheets/pdf_di...    2018.06.25   
1  http://sharkattackfile.net/spreadsheets/pdf_di...    2018.06.18   

  case_number.2 original_order unnamed:_22 unnamed:_23  
0    2018.06.25         6303.0         NaN         NaN  
1    2018.06.18         6302.0         NaN         NaN  

[2 rows x 24 columns]

ELIMINO COLUMNAS y FILAS que no me aportan valor.

1- COLUMNAS

Elimino columnas que no aportan nada por significado.

In [26]:
attacks.drop(["pdf", "href_formula","href","unnamed:_22", "unnamed:_23"],axis=1,inplace=True)

In [27]:
attacks.head(2)

case_number         date    year        type country        area  \
0  2018.06.25  25-Jun-2018  2018.0     Boating     USA  California   
1  2018.06.18  18-Jun-2018  2018.0  Unprovoked     USA     Georgia   

                         location  activity             name sex_ age  \
0     Oceanside, San Diego County  Paddling      Julie Wolfe    F  57   
1  St. Simon Island, Glynn County  Standing  Adyson McNeely     F  11   

                                              injury fatal_           time  \
0  No injury to occupant, outrigger canoe and pad...      N          18h00   
1                         Minor injury to left thigh      N  14h00  -15h00   

      species_          investigator_or_source case_number.1 case_number.2  \
0  White shark                R. Collier, GSAF    2018.06.25    2018.06.25   
1          NaN  K.McMurray, TrackingSharks.com    2018.06.18    2018.06.18   

   original_order  
0          6303.0  
1          6302.0

Aquí, elimino columnas que no aportan nada a mis hipótesis.

In [28]:
attacks.drop(["time", "species_","investigator_or_source","case_number.1", "case_number.2", "original_order"],axis=1,inplace=True)

In [29]:
attacks.head(2)

case_number         date    year        type country        area  \
0  2018.06.25  25-Jun-2018  2018.0     Boating     USA  California   
1  2018.06.18  18-Jun-2018  2018.0  Unprovoked     USA     Georgia   

                         location  activity             name sex_ age  \
0     Oceanside, San Diego County  Paddling      Julie Wolfe    F  57   
1  St. Simon Island, Glynn County  Standing  Adyson McNeely     F  11   

                                              injury fatal_  
0  No injury to occupant, outrigger canoe and pad...      N  
1                         Minor injury to left thigh      N

2-FILAS.

Elimino las filas en las que sus valores son todo NaN; no aporta nada. 

Para este paso he cambiado de data frame por si metía la pata. A apartir de aquí trabajo con el data frame "sharks"

In [30]:
sharks = attacks.dropna(how="all", axis=0)

In [31]:
sharks.shape

(8703, 13)

# LIMPIEZA por columnas.
COLUMNA; case_number

Elimino los ".letra" de todos los case_number. Aunque no aporta much valor, como hice ya esta limpieza, lo dejo indicado.

In [32]:
case_number = list(attacks.case_number.unique())

In [33]:
"""def quitar_a(x):
    if ".a" in str(x):
        return x.replace(".a", "")
    else:
        return x"""

'def quitar_a(x):\n    if ".a" in str(x):\n        return x.replace(".a", "")\n    else:\n        return x'

In [34]:
quitar_a

<function src.cleaning.quitar_a(x)>

In [35]:
sharks["case_number"] = sharks.case_number.apply(quitar_a)

<ipython-input-35-b9e4096ae751>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_a)


In [36]:
"""def quitar_b(x):
    if ".b" in str(x):
        return x.replace(".b", "")
    else:
        return x"""

'def quitar_b(x):\n    if ".b" in str(x):\n        return x.replace(".b", "")\n    else:\n        return x'

In [37]:
quitar_b

<function src.cleaning.quitar_b(x)>

In [38]:
sharks["case_number"] = sharks.case_number.apply(quitar_b)

<ipython-input-38-7a0d2429a33b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_b)


In [39]:
"""def quitar_c(x):
    if ".c" in str(x):
        return x.replace(".c", "")
    else:
        return x"""

'def quitar_c(x):\n    if ".c" in str(x):\n        return x.replace(".c", "")\n    else:\n        return x'

In [40]:
quitar_c

<function src.cleaning.quitar_c(x)>

In [41]:
sharks["case_number"] = sharks.case_number.apply(quitar_c)

<ipython-input-41-560dba157aa4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_c)


In [42]:
"""def quitar_d(x):
    if ".c" in str(x):
        return x.replace(".d", "")
    else:
        return x"""

'def quitar_d(x):\n    if ".c" in str(x):\n        return x.replace(".d", "")\n    else:\n        return x'

In [43]:
quitar_d

<function src.cleaning.quitar_d(x)>

In [44]:
sharks["case_number"] = sharks.case_number.apply(quitar_d)

<ipython-input-44-327631ea2d19>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_d)


In [45]:
"""def quitar_R(x):
    if ".R" in str(x):
        return x.replace(".R", "")
    else:
        return x"""

'def quitar_R(x):\n    if ".R" in str(x):\n        return x.replace(".R", "")\n    else:\n        return x'

In [46]:
quitar_R

<function src.cleaning.quitar_R(x)>

In [47]:
sharks["case_number"] = sharks.case_number.apply(quitar_R)

<ipython-input-47-46223fb203ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["case_number"] = sharks.case_number.apply(quitar_R)


COLUMNA; date

Para la columna "date" me quedo solo con fecha y año. 

In [48]:
sharks['newcol'] = sharks['date'].str.extract(r"(\w+-\d+)")

<ipython-input-48-65ad4d2e00fd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks['newcol'] = sharks['date'].str.extract(r"(\w+-\d+)")


In [49]:
sharks.sample(2)

case_number         date    year        type       country  \
3413  1970.00.00        1970s  1970.0  Unprovoked          IRAQ   
2971  1982.07.31  31-Jul-1982  1982.0     Invalid  SOUTH AFRICA   

               area                                location  \
3413         Basrah  Shatt-al-Arab River near Abu al Khasib   
2971  KwaZulu-Natal                             Amanzimtoti   

                       activity                name sex_ age  \
3413  Washing clothes on stairs              female    F  25   
2971            Boogie boarding  Riaan van Rensburg    M  20   

                          injury fatal_    newcol  
3413           Left foot severed      N       NaN  
2971  No injury, swim fin bitten    NaN  Jul-1982

In [50]:
dates = list(sharks.date)
print(dates[0:5])

['25-Jun-2018', '18-Jun-2018', '09-Jun-2018', '08-Jun-2018', '04-Jun-2018']


In [51]:
newcoll= list(sharks.newcol)
print(newcoll[0:5])

['Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018', 'Jun-2018']


In [96]:
sharks.date=newcoll

<ipython-input-96-af56e7fbce42>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  sharks.dates=newcoll


In [100]:
# Compruebo que me ha cambiado las fechas.
print(sharks.date[0:5])

0    Jun-2018
1    Jun-2018
2    Jun-2018
3    Jun-2018
4    Jun-2018
Name: date, dtype: object


In [54]:
sharks.sample()

case_number date  year type country area location activity name sex_  \
6629           0  NaN   NaN  NaN     NaN  NaN      NaN      NaN  NaN  NaN   

      age injury fatal_ newcol  
6629  NaN    NaN    NaN    NaN

In [55]:
# Como he cambiado la date, puedo eliminar ya newcol
sharks.drop(["newcol"],axis=1, inplace=True)

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [56]:
sharks.sample()

case_number      date    year      type country        area  \
4982  1932.08.10  Aug-1932  1932.0  Provoked     USA  New Jersey   

                                               location activity        name  \
4982  Offshore, between Mantoloking & Point Pleasant...  Fishing  John Olsen   

     sex_  age                                             injury fatal_  
4982    M  NaN  Hooked shark gashed right leg      PROVOKED IN...      N

COLUMNA; year

Elimino el ".0" de los años.

In [57]:
"""def year_(x):
    if ".0" in str(x):
        return str(x).replace(".0", "")
    else:
        return x"""

'def year_(x):\n    if ".0" in str(x):\n        return str(x).replace(".0", "")\n    else:\n        return x'

In [58]:
year_

<function src.cleaning.year_(x)>

In [59]:
sharks["year"] = sharks.year.apply(year_)

<ipython-input-59-34703d824513>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["year"] = sharks.year.apply(year_)


In [60]:
print(sharks.year)

0        2018
1        2018
2        2018
3        2018
4        2018
         ... 
8698      NaN
8699      NaN
8700      NaN
8701      NaN
25722     NaN
Name: year, Length: 8703, dtype: object


In [61]:
# Este es el listado final de todas mis años.
sharks.year.unique()

array(['2018', '2017', nan, '2016', '2015', '2014', '2013', '2012',
       '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004',
       '2003', '2002', '2001', '2000', '1999', '1998', '1997', '1996',
       '1995', '1984', '1994', '1993', '1992', '1991', '1990', '1989',
       '1969', '1988', '1987', '1986', '1985', '1983', '1982', '1981',
       '1980', '1979', '1978', '1977', '1976', '1975', '1974', '1973',
       '1972', '1971', '1970', '1968', '1967', '1966', '1965', '1964',
       '1963', '1962', '1961', '1960', '1959', '1958', '1957', '1956',
       '1955', '1954', '1953', '1952', '1951', '1950', '1949', '1948',
       '1848', '1947', '1946', '1945', '1944', '1943', '1942', '1941',
       '1940', '1939', '1938', '1937', '1936', '1935', '1934', '1933',
       '1932', '1931', '1930', '1929', '1928', '1927', '1926', '1925',
       '1924', '1923', '1922', '1921', '1920', '1919', '1918', '1917',
       '1916', '1915', '1914', '1913', '1912', '1911', '1910', '1909',
       '1

COLUMNA type.

Cambio nombre de "Unprovoked" por "Accidental".Además he metido "Boatomg" y "Boating" en una sola columna "Boat" 

In [62]:
sharks.type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

In [63]:
"""def type_(x):
    if "Unprovoked" in str(x):
        return x.replace("Unprovoked", "Accidental")
    else:
        return x"""

'def type_(x):\n    if "Unprovoked" in str(x):\n        return x.replace("Unprovoked", "Accidental")\n    else:\n        return x'

In [64]:
type_

<function src.cleaning.type_(x)>

In [65]:
sharks["type"] = sharks.type.apply(type_)

<ipython-input-65-e5a48b295325>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_)


In [66]:
multi_p = sharks[(sharks.type == "Boatomg")]

In [67]:
multi_p

case_number      date  year     type      country          area  \
1152  2009.03.01  Mar-2009  2009  Boatomg  NEW ZEALAND  North Island   

      location activity                                             name sex_  \
1152  Taranaki  Fishing  boat, occupants: Boyd Rutherford & Hamish Roper    M   

      age                                       injury fatal_  
1152  NaN  No injury to occupants, shark bit propeller      N

In [68]:
"""def type_1(x):
    if "Boatomg" in str(x):
        return x.replace("Boatomg", "Boat")
    else:
        return x"""

'def type_1(x):\n    if "Boatomg" in str(x):\n        return x.replace("Boatomg", "Boat")\n    else:\n        return x'

In [69]:
type_1

<function src.cleaning.type_1(x)>

In [70]:
sharks["type"] = sharks.type.apply(type_1)

<ipython-input-70-240476cb040b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_1)


In [71]:
"""def type_2(x):
    if "Invalid" in str(x):
        return x.replace("Invalid", "Injured/Fatal")
    else:
        return x"""

'def type_2(x):\n    if "Invalid" in str(x):\n        return x.replace("Invalid", "Injured/Fatal")\n    else:\n        return x'

In [72]:
type_2

<function src.cleaning.type_2(x)>

In [73]:
sharks["type"] = sharks.type.apply(type_2)

<ipython-input-73-0df61136e9dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_2)


In [74]:
"""def type_3(x):
    if "Boating" in str(x):
        return x.replace("Boating", "Boat")
    else:
        return x"""

'def type_3(x):\n    if "Boating" in str(x):\n        return x.replace("Boating", "Boat")\n    else:\n        return x'

In [75]:
type_3

<function src.cleaning.type_3(x)>

In [76]:
sharks["type"] = sharks.type.apply(type_3)

<ipython-input-76-75c94eba09d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["type"] = sharks.type.apply(type_3)


In [77]:
sharks.type.unique()

array(['Boat', 'Accidental', 'Injured/Fatal', 'Provoked', 'Questionable',
       'Sea Disaster', nan], dtype=object)

COLUMNA; Pais, área y localización

Área y localización no me afecta a mis hipótesis; no los toco aunque se puedan mejorar

He pasado todos los países a mayúsculas y eliminado "?". He metido también "ENGLAND" en "UNITED KINGDOM"

In [78]:
countries = list(sharks.country.unique())

In [79]:
len(countries)

213

In [80]:
print(countries[0:20])

['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA', 'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA', 'ECUADOR', 'MALAYSIA', 'LIBYA', nan, 'CUBA', 'MAURITIUS', 'NEW ZEALAND', 'SPAIN', 'SAMOA']


Ponemos todos los países en mayúsculas.

In [81]:
"""def upper_c(x):
    return str(x).upper()"""

'def upper_c(x):\n    return str(x).upper()'

In [82]:
upper_c

<function src.cleaning.upper_c(x)>

In [83]:
sharks["country"] = sharks.country.apply(upper_c)

<ipython-input-83-51f75b38b595>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(upper_c)


Reemplazar ENGLAND por UNITED KINGDOM

In [84]:
"""def england_(x):
    if "ENGLAND" in str(x):
        return x.replace("ENGLAND", "UNITED KINGDOM")
    else:
        return x"""

'def england_(x):\n    if "ENGLAND" in str(x):\n        return x.replace("ENGLAND", "UNITED KINGDOM")\n    else:\n        return x'

In [85]:
england_

<function src.cleaning.england_(x)>

In [86]:
sharks["country"] = sharks.country.apply(england_)

<ipython-input-86-9a5641fac9da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(england_)


ELIMINO signo de interrogación.

In [87]:
"""def question_(x):
    if "?" in str(x):
        return x.replace("?", "")
    else:
        return x"""

'def question_(x):\n    if "?" in str(x):\n        return x.replace("?", "")\n    else:\n        return x'

In [88]:
question_

<function src.cleaning.question_(x)>

In [89]:
sharks["country"] = sharks.country.apply(question_)

<ipython-input-89-1d540e1a7bd3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["country"] = sharks.country.apply(question_)


In [90]:
# Este es el listado final de todos mis paises.
sharks.country.unique()

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'UNITED KINGDOM',
       'SOUTH AFRICA', 'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS',
       'NEW CALEDONIA', 'ECUADOR', 'MALAYSIA', 'LIBYA', 'NAN', 'CUBA',
       'MAURITIUS', 'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS',
       'JAPAN', 'EGYPT', 'ST HELENA, BRITISH OVERSEAS TERRITORY',
       'COMOROS', 'REUNION', 'FRENCH POLYNESIA', 'UNITED ARAB EMIRATES',
       'PHILIPPINES', 'INDONESIA', 'CHINA', 'COLUMBIA', 'CAPE VERDE',
       'FIJI', 'DOMINICAN REPUBLIC', 'CAYMAN ISLANDS', 'ARUBA',
       'MOZAMBIQUE', 'PUERTO RICO', 'ITALY', 'ATLANTIC OCEAN', 'GREECE',
       'ST. MARTIN', 'FRANCE', 'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO',
       'KIRIBATI', 'ISRAEL', 'DIEGO GARCIA', 'TAIWAN', 'JAMAICA',
       'PALESTINIAN TERRITORIES', 'GUAM', 'SEYCHELLES', 'BELIZE',
       'NIGERIA', 'TONGA', 'SCOTLAND', 'CANADA', 'CROATIA',
       'SAUDI ARABIA', 'CHILE', 'ANTIGUA', 'KENYA', 'RUSSIA',
       'TURKS & CAICOS', 'UNITED ARAB EMIRATES (

COLUMNA; Activity

Aunque mis hipótesis afectan a actividades, para validarlo he optado por otra opción que no sea limpieza. Ver validación.

In [101]:
activities = list(sharks.activity.unique())

In [102]:
activities

['Paddling',
 'Standing',
 'Surfing',
 'Free diving',
 'Kite surfing',
 'Swimming',
 'Fishing',
 'Walking',
 'Feeding sharks',
 'Boogie boarding',
 'Scuba diving',
 'Paddle-skiing',
 'Body boarding',
 'Windsurfing',
 'Stand-Up Paddleboarding',
 'Wading',
 'Scuba Diving',
 'Kayak fishing for sharks',
 'Cleaning fish',
 'Snorkeling',
 'Spearfishing',
 nan,
 'Diving',
 'Fishing / Wading',
 '2 boats capsized',
 'Night bathing',
 'Surfing ',
 'Kayaking / Fishing',
 'Kayaking',
 'Body surfing',
 'Swimming, poaching abalone',
 'Canoeing',
 'SUP',
 'Skimboarding',
 'Touching a shark',
 'Fishing ',
 'Attempting to lasso a shark',
 'Photo shoot',
 'Paddle boarding',
 'Kakaying',
 'Washing hands',
 'Grabbing shark for a selfie',
 'Kayak fishing',
 'Tagging sharks',
 'Surf skiing ',
 'Surf fishing',
 'Floating',
 'Cage Diving',
 'Surfng',
 'SUP Foil boarding',
 'Lobstering',
 'Fishing in Alabama Deep Fishing Rodeo',
 'Fishing for squid',
 'Fishing for sharks',
 'Floating in tube',
 'Teasing a shar

COLUMNA: sex_

Limpio para quedarme sólo con F, M o None

In [103]:
gender = list(sharks.sex_.unique())

In [104]:
gender

['F', 'M', nan, 'M ', 'lli', 'N', '.']

In [105]:
sharks.head(2)

case_number      date  year        type country        area  \
0  2018.06.25  Jun-2018  2018        Boat     USA  California   
1  2018.06.18  Jun-2018  2018  Accidental     USA     Georgia   

                         location  activity             name sex_ age  \
0     Oceanside, San Diego County  Paddling      Julie Wolfe    F  57   
1  St. Simon Island, Glynn County  Standing  Adyson McNeely     F  11   

                                              injury fatal_  
0  No injury to occupant, outrigger canoe and pad...      N  
1                         Minor injury to left thigh      N

In [106]:
"""def gender_(x):
    special_characters = 'FM'
    if str(x) in special_characters:
        return x
    else:
        return str(x).replace(str(x), "None")"""

'def gender_(x):\n    special_characters = \'FM\'\n    if str(x) in special_characters:\n        return x\n    else:\n        return str(x).replace(str(x), "None")'

In [107]:
gender_

<function src.cleaning.gender_(x)>

In [108]:
sharks["genero"] = sharks.sex_.apply(gender_)

<ipython-input-108-0220de8c0124>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["genero"] = sharks.sex_.apply(gender_)


In [109]:
sharks.head()

case_number      date  year           type    country             area  \
0  2018.06.25  Jun-2018  2018           Boat        USA       California   
1  2018.06.18  Jun-2018  2018     Accidental        USA          Georgia   
2  2018.06.09  Jun-2018  2018  Injured/Fatal        USA           Hawaii   
3  2018.06.08  Jun-2018  2018     Accidental  AUSTRALIA  New South Wales   
4  2018.06.04  Jun-2018  2018       Provoked     MEXICO           Colima   

                         location     activity             name sex_  age  \
0     Oceanside, San Diego County     Paddling      Julie Wolfe    F   57   
1  St. Simon Island, Glynn County     Standing  Adyson McNeely     F   11   
2                    Habush, Oahu      Surfing      John Denges    M   48   
3              Arrawarra Headland      Surfing             male    M  NaN   
4                        La Ticla  Free diving   Gustavo Ramos     M  NaN   

                                              injury fatal_ genero  
0  No injury to occupant, outrigger canoe and pad...      N      F  
1                         Minor injury to left thigh      N      F  
2       Injury to left lower leg from surfboard skeg      N      M  
3                          Minor injury to lower leg      N      M  
4  Lacerations to leg & hand shark PROVOKED INCIDENT      N      M

In [110]:
gender = list(sharks.genero.unique())

In [111]:
gender

['F', 'M', 'None']

In [112]:
sharks.drop(["sex_"],axis=1,inplace=True)

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [113]:
sharks.head()

case_number      date  year           type    country             area  \
0  2018.06.25  Jun-2018  2018           Boat        USA       California   
1  2018.06.18  Jun-2018  2018     Accidental        USA          Georgia   
2  2018.06.09  Jun-2018  2018  Injured/Fatal        USA           Hawaii   
3  2018.06.08  Jun-2018  2018     Accidental  AUSTRALIA  New South Wales   
4  2018.06.04  Jun-2018  2018       Provoked     MEXICO           Colima   

                         location     activity             name  age  \
0     Oceanside, San Diego County     Paddling      Julie Wolfe   57   
1  St. Simon Island, Glynn County     Standing  Adyson McNeely    11   
2                    Habush, Oahu      Surfing      John Denges   48   
3              Arrawarra Headland      Surfing             male  NaN   
4                        La Ticla  Free diving   Gustavo Ramos   NaN   

                                              injury fatal_ genero  
0  No injury to occupant, outrigger canoe and pad...      N      F  
1                         Minor injury to left thigh      N      F  
2       Injury to left lower leg from surfboard skeg      N      M  
3                          Minor injury to lower leg      N      M  
4  Lacerations to leg & hand shark PROVOKED INCIDENT      N      M

COLUMNA: fatal_

Limpio para quedarme con N, Y o None

In [114]:
fatal = list(sharks.fatal_.unique())

In [115]:
fatal

['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y']

In [116]:
"""def fatal_(x):
    special_characters = 'NYy'
    if str(x) in special_characters:
        return x.upper()
    else:
        return str(x).replace(str(x), "None")"""

'def fatal_(x):\n    special_characters = \'NYy\'\n    if str(x) in special_characters:\n        return x.upper()\n    else:\n        return str(x).replace(str(x), "None")'

In [117]:
fatal_

<function src.cleaning.fatal_(x)>

In [118]:
sharks["fatal"] = sharks.fatal_.apply(fatal_)

<ipython-input-118-988055907673>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["fatal"] = sharks.fatal_.apply(fatal_)


In [119]:
sharks.head(2)

case_number      date  year        type country        area  \
0  2018.06.25  Jun-2018  2018        Boat     USA  California   
1  2018.06.18  Jun-2018  2018  Accidental     USA     Georgia   

                         location  activity             name age  \
0     Oceanside, San Diego County  Paddling      Julie Wolfe  57   
1  St. Simon Island, Glynn County  Standing  Adyson McNeely   11   

                                              injury fatal_ genero fatal  
0  No injury to occupant, outrigger canoe and pad...      N      F     N  
1                         Minor injury to left thigh      N      F     N

In [120]:
fatal_ = list(sharks.fatal.unique())

In [121]:
fatal_

['N', 'Y', 'None']

In [122]:
sharks.drop(["fatal_"],axis=1,inplace=True)

/home/jesus_chao/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [123]:
sharks.head(2)

case_number      date  year        type country        area  \
0  2018.06.25  Jun-2018  2018        Boat     USA  California   
1  2018.06.18  Jun-2018  2018  Accidental     USA     Georgia   

                         location  activity             name age  \
0     Oceanside, San Diego County  Paddling      Julie Wolfe  57   
1  St. Simon Island, Glynn County  Standing  Adyson McNeely   11   

                                              injury genero fatal  
0  No injury to occupant, outrigger canoe and pad...      F     N  
1                         Minor injury to left thigh      F     N

# Validación HIPOTESIS

1- USA es donde más accidentes de tiburón se producen.

Cuento los valores de la columna para validar

In [124]:
sharks.country.value_counts()

NAN                      2451
USA                      2229
AUSTRALIA                1338
SOUTH AFRICA              579
PAPUA NEW GUINEA          134
                         ... 
DIEGO GARCIA                1
ARGENTINA                   1
THE BALKANS                 1
NORTH ATLANTIC OCEAN        1
CEYLON (SRI LANKA)          1
Name: country, Length: 206, dtype: int64

2- La mayor parte de los accidentes son no provocados.

Cuento los valores de la columna para validar.

In [125]:
sharks.type.value_counts()

Accidental       4595
Provoked          574
Injured/Fatal     547
Boat              341
Sea Disaster      239
Questionable        2
Name: type, dtype: int64

3- Fishing y Surfing; mayores causas de accidentes.

Creo dos columnas Tue or False para datos que terminan en "ishing" y "urfing". Los cuento y compruebo la hipótesis.

In [126]:
sharks["fishing"] = sharks["activity"].str.contains("ishing")

<ipython-input-126-bfcf38036315>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["fishing"] = sharks["activity"].str.contains("ishing")


In [127]:
sharks["surfing"] = sharks["activity"].str.contains("urfing")

<ipython-input-127-d92449e05581>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks["surfing"] = sharks["activity"].str.contains("urfing")


In [128]:
sharks.head(2)

case_number      date  year        type country        area  \
0  2018.06.25  Jun-2018  2018        Boat     USA  California   
1  2018.06.18  Jun-2018  2018  Accidental     USA     Georgia   

                         location  activity             name age  \
0     Oceanside, San Diego County  Paddling      Julie Wolfe  57   
1  St. Simon Island, Glynn County  Standing  Adyson McNeely   11   

                                              injury genero fatal fishing  \
0  No injury to occupant, outrigger canoe and pad...      F     N   False   
1                         Minor injury to left thigh      F     N   False   

  surfing  
0   False  
1   False

In [129]:
sharks.fishing.value_counts()

False    4586
True     1172
Name: fishing, dtype: int64

In [130]:
sharks.surfing.value_counts()

False    4630
True     1128
Name: surfing, dtype: int64

In [131]:
sharks.isnull().sum()

case_number       1
date           2946
year           2403
type           2405
country           0
area           2856
location       2941
activity       2945
name           2611
age            5232
injury         2429
genero            0
fatal             0
fishing        2945
surfing        2945
dtype: int64

2945 nulls

In [132]:
suma_fish_surf = 1128 + 1172

In [133]:
Porcentaje = suma_fish_surf/(4630+1128)

In [134]:
Porcentaje

0.399444251476207

4- Mueren más hombres que mujeres.

Cuento los valores de la columna para validar.

In [135]:
hombres_muertos = sharks[(sharks.genero == "M") & (sharks.fatal == "Y")]

In [136]:
hombres_muertos.genero.value_counts()

M    1173
Name: genero, dtype: int64

In [137]:
mujeres_muertas = sharks[(sharks.genero == "F") & (sharks.fatal == "Y")]

In [138]:
mujeres_muertas.genero.value_counts()

F    107
Name: genero, dtype: int64

# Conclusión

Las HIPOTESIS 1, 2, y 4 son verdaderas la 3 es falsa.

Esta es el aspecto final del csv

In [139]:
sharks.head(3)

case_number      date  year           type country        area  \
0  2018.06.25  Jun-2018  2018           Boat     USA  California   
1  2018.06.18  Jun-2018  2018     Accidental     USA     Georgia   
2  2018.06.09  Jun-2018  2018  Injured/Fatal     USA      Hawaii   

                         location  activity             name age  \
0     Oceanside, San Diego County  Paddling      Julie Wolfe  57   
1  St. Simon Island, Glynn County  Standing  Adyson McNeely   11   
2                    Habush, Oahu   Surfing      John Denges  48   

                                              injury genero fatal fishing  \
0  No injury to occupant, outrigger canoe and pad...      F     N   False   
1                         Minor injury to left thigh      F     N   False   
2       Injury to left lower leg from surfboard skeg      M     N   False   

  surfing  
0   False  
1   False  
2    True

Exporto CSV limpio.

In [140]:
sharks.to_csv("sharks.csv")